## SoundCloud likes downloader

#### Modules Required

In [ ]:
!pip3 install selenium
!pip3 install pandas

#### What do I need to accomplish?

- Download the songs 
- Build data dictionary for the 1k songs

````
example of data dictionary
 {
          name: "Name of Song",
          artist: "Artist Name",
          cover: "https://raw.githubusercontent.com/myrepo/myart.jpg",
          source: "https://raw.githubusercontent.com/myrepo/song.mp3",
          url: "SoundCloudSongLink",
          favorited: false
}

````

#### Looking at dependencies.

In [8]:
import pandas as pd

#### Taking look at dataset

In [9]:
songs = pd.read_csv('songs.csv')
songs.head()

,web-scraper-order,web-scraper-start-url,songs,songs-href
0,1628348290-1681,https://soundcloud.com/you/likes,⛧ SUNBURST ⛧,https://soundcloud.com/kaivaan/sunburst
1,1628348290-888,https://soundcloud.com/you/likes,Otafunk!,https://soundcloud.com/imyourlovergirl/otafunk
2,1628348290-1119,https://soundcloud.com/you/likes,Plastic Memories Ending (fb: HARDANIME),https://soundcloud.com/hardanime-fb/plastic-me...
3,1628348290-1008,https://soundcloud.com/you/likes,Diana's Heart,https://soundcloud.com/maisonorange/dianas-heart
4,1628348290-1615,https://soundcloud.com/you/likes,Grant - Are We Still Young (feat. Jessi Mason),https://soundcloud.com/grantsongs/grant-are-we...


#### Web Engine with Selenium

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import urllib.request

def GetSongNameFromURL(url):
	url = url[1].split('/')
	return url[len(url)-1]	

metadata = {"name":[],"cover":[],"source":[],"url":[],"favorited":[]}

def DownloadFile(downloadurl,filename,imageurl,soundcloudurl):
	bPass = False
	try:
		urllib.request.urlretrieve(downloadurl, "music\{0}.mp3".format(filename))
		bPass = True
		
	except Exception as e:
		bPass = False
		print("Error",e)

	if bPass == True:
		print("downloaded {0}.mp3".format(filename))
		metadata['name'].append(filename)
		metadata['cover'].append(imageurl)
		metadata['source'].append("https://github.com/ADGVLOGS/ADGMusicPlayer/blob/main/music/"+filename+".mp3?raw=true")
		metadata['url'].append(soundcloudurl[1])
		metadata['favorited'].append("false")
#Our Engine
def GetSoundCloudMP3LinkRawEngine(soundcloudurls):
	#look here https://chromedriver.storage.googleapis.com/index.html?path=92.0.4515.107/for driver downloads based on operating system it has to be path of executable
	urls = [] 
	driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
	for url in soundcloudurls.items():
		bPass = False
		try:
			driver.get("https://sclouddownloader.net/")
			search_bar = driver.find_element_by_name("sound-url")
			search_bar.clear()
			search_bar.send_keys(url[1])
			search_bar.send_keys(Keys.RETURN)
			links=driver.find_elements_by_tag_name("a")
			imageurl = driver.find_element_by_id("thumbnail").get_attribute("src")
			urls.append(links[5].get_attribute("href"))
			songart = GetSongNameFromURL(url)
			bPass = True
			
		except Exception as e:
			bPass = False
			print("Error",e)
		
		if bPass == True:
			DownloadFile(links[5].get_attribute("href"),songart,imageurl,url)

	
	return urls
	
	

#### Our Data Structures

In [11]:
#dictionary objects
songtitle = dict(songs['songs'])
songurls = dict(songs['songs-href'])
urls = GetSoundCloudMP3LinkRawEngine(songurls)


downloaded sunburst.mp3
downloaded otafunk.mp3
downloaded plastic-memories-ending-fb-hardanime.mp3
downloaded dianas-heart.mp3
downloaded grant-are-we-still-young.mp3
downloaded elevation_of_love.mp3
downloaded connection.mp3
downloaded love-in-the-midnight.mp3
downloaded 100-production-mix-1.mp3
downloaded virgins-club.mp3
Error HTTP Error 403: Forbidden
downloaded youve-got-me-yung-bae.mp3
downloaded lokan-antigravity.mp3
downloaded roblox-type-beat.mp3
downloaded departure.mp3
downloaded pyre.mp3
downloaded girls-last-groove.mp3
downloaded freeze.mp3
downloaded porter-robinson-madeon-shelter-5.mp3
downloaded crossing-faygo-v3.mp3
downloaded rei-stormy-weather.mp3
downloaded duck-song-ft-jamil.mp3
downloaded ice-cream.mp3
downloaded conro-all-me.mp3
downloaded down.mp3
downloaded contaminate.mp3
downloaded corporate-japan.mp3
downloaded life-of-a-hentai-dude.mp3
downloaded omniboi-gucci.mp3
downloaded rome-in-silver-friends-feat-chae.mp3
downloaded koven-light-up.mp3
downloaded child

In [2]:
import pandas as pd
df = pd.DataFrame.from_dict(metadata)
def GetArtistNameFromURL(url):
	url = url.split('/')
	return url[len(url)-2].replace('-',' ').title()	

artist = []
for index, row in df.iterrows():
	row['name'] = row['name'].replace('-',' ').title()
	artist.append(GetArtistNameFromURL(row['url']))

df['artist'] = artist
df.to_csv("MusicDBTemp.csv") 
print("CSV File Saved Success")
df.head()

NameError: name 'metadata' is not defined

#### Removing files > 25mb

In [6]:
import os
rootdir = 'music/'

songsizes = {"songfile":[],"Size (MB)":[]}
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        songsizes["songfile"].append(os.path.join(file))
        songsizes["Size (MB)"].append(round(os.path.getsize(rootdir+file) / (1024*1024),2))
        
dfmetadata = pd.DataFrame.from_dict(songsizes)
dfmetadata.to_csv("songmetadata.csv")
print("Total Size of libary is",dfmetadata['Size (MB)'].sum(),"MB")
dfmetadata


Total Size of libary is 3089.1099999999997 MB


,songfile,Size (MB)
0,01-and-then-a-creative-mind-was-born.mp3,1.62
1,01-easy-love.mp3,4.01
2,01-koi-no-orchestra.mp3,3.92
3,02-ikasuze-positive-thinking.mp3,3.01
4,02-umi-ga-kikoeru.mp3,5.53
...,...,...
945,zac-efrons-squeaky-surfboard-click-buy-for-dl.mp3,4.20
946,zebulon.mp3,4.92
947,zef-goema-megamix.mp3,10.74
948,zoot-sims-mixed-emotions.mp3,2.33


#### Building database of what we have already

In [7]:
dfmusic = pd.read_csv("MusicDBTemp.csv")
dfmusic['songfile'] = dfmetadata['songfile']
dfmusic['Size (MB)'] = dfmetadata['Size (MB)']
del dfmusic['Unnamed: 0']
dfmusic.to_csv("FullDB.csv")
print("Database created","FullDB.csv")

print("Size of music database is",dfmusic['Size (MB)'].sum(),"MB")
print("We have",dfmusic.shape[0],"songs")
dfmusic.head()

Database created FullDB.csv
Size of music database is 3089.1099999999997 MB
We have 985 songs


,name,cover,source,url,favorited,artist,songfile,Size (MB)
0,Sunburst,https://i1.sndcdn.com/artworks-OLjT4Jym05UzJLb...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/kaivaan/sunburst,False,Kaivaan,01-and-then-a-creative-mind-was-born.mp3,1.62
1,Otafunk,https://i1.sndcdn.com/artworks-eJ5CAlhKQtgWyp0...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/imyourlovergirl/otafunk,False,Imyourlovergirl,01-easy-love.mp3,4.01
2,Plastic Memories Ending Fb Hardanime,https://i1.sndcdn.com/artworks-000119418000-7z...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/hardanime-fb/plastic-me...,False,Hardanime Fb,01-koi-no-orchestra.mp3,3.92
3,Dianas Heart,https://i1.sndcdn.com/artworks-acy8yVKyIetmycY...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/maisonorange/dianas-heart,False,Maisonorange,02-ikasuze-positive-thinking.mp3,3.01
4,Grant Are We Still Young,https://i1.sndcdn.com/artworks-000224494934-ap...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/grantsongs/grant-are-we...,False,Grantsongs,02-umi-ga-kikoeru.mp3,5.53


#### Github Restrictions

<p>Since i am not allowed to upload more than 25mb per file for .mp3 files what i am going to do remove the files from database which is > 25mb so below is function</p>

In [8]:
import os

count = 0 
for index, row in dfmusic.iterrows():
    if row['Size (MB)'] > 25:
        count = count + 1
        dfmusic.drop(index, inplace=True)
        if os.path.exists("music\{0}".format(row['songfile'])):
             os.remove("music\{0}".format(row['songfile']))
             print(row['songfile'],'removed')

print("Size of music database is",dfmusic['Size (MB)'].sum(),"MB")
print("We have",dfmusic.shape[0],"songs")
print(count,'songs removed')
dfmusic.to_csv("GitHubMusicDatabase.csv")
print('database created')
dfmusic.head()

Size of music database is 3089.1099999999997 MB
We have 985 songs
0 songs removed
database created


,name,cover,source,url,favorited,artist,songfile,Size (MB)
0,Sunburst,https://i1.sndcdn.com/artworks-OLjT4Jym05UzJLb...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/kaivaan/sunburst,False,Kaivaan,01-and-then-a-creative-mind-was-born.mp3,1.62
1,Otafunk,https://i1.sndcdn.com/artworks-eJ5CAlhKQtgWyp0...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/imyourlovergirl/otafunk,False,Imyourlovergirl,01-easy-love.mp3,4.01
2,Plastic Memories Ending Fb Hardanime,https://i1.sndcdn.com/artworks-000119418000-7z...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/hardanime-fb/plastic-me...,False,Hardanime Fb,01-koi-no-orchestra.mp3,3.92
3,Dianas Heart,https://i1.sndcdn.com/artworks-acy8yVKyIetmycY...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/maisonorange/dianas-heart,False,Maisonorange,02-ikasuze-positive-thinking.mp3,3.01
4,Grant Are We Still Young,https://i1.sndcdn.com/artworks-000224494934-ap...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/grantsongs/grant-are-we...,False,Grantsongs,02-umi-ga-kikoeru.mp3,5.53


#### Dropping rows which returns 403 errors for songs in GitHub Database 

In [14]:
import urllib.request
for index,data in dfmusic.iterrows():
	try:
		response = urllib.request.urlopen(data['source'])
		status_code = response.getcode()
		
	except:
		dfmusic.drop(index, inplace=True)
		print("dropping",data['songfile'])	

dropping 01-koi-no-orchestra.mp3
dropping 02-ikasuze-positive-thinking.mp3
dropping 22-jaq-world-orca.mp3
dropping 3-6-destruction.mp3
dropping 3pm.mp3
dropping a-mere-machine.mp3
dropping ano-hana-ending-secret-base.mp3
dropping at-the-lovers-p-o-o-l-p-a-r-t-y-night.mp3
dropping beach.mp3
dropping bedtime-stories.mp3
dropping blind_faith.mp3
dropping bounce-devi-deep-house-remix.mp3
dropping breezin.mp3
dropping broke-n.mp3
dropping chikatto-chika-chika.mp3
dropping chobits-let-me-be-with-you.mp3
dropping chocolate-groove.mp3
dropping christmas.mp3
dropping conscious.mp3
dropping continue.mp3
dropping cookieekawaii-vibe-part-2-prod.mp3
dropping cors-moon-garden-ghost-data-remix.mp3
dropping cosmic-nocturne.mp3
dropping dance-in-the-memories-english.mp3
dropping def-pon-me-many-men.mp3
dropping delicious.mp3
dropping departure.mp3
dropping dick-sweat-on-my-wrist.mp3
dropping difficulties.mp3
dropping diode.mp3
dropping dj-ashlins-house-mix.mp3
dropping dj-snake-ultra-miami-2016-live.mp

#### To randomly shuffle the list

In [22]:
dfmusic = dfmusic.sample(frac=1).reset_index(drop=True)
dfmusic.head()

,name,cover,source,url,favorited,artist,songfile,Size (MB)
0,Godspeed,https://i1.sndcdn.com/artworks-000243646858-e6...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/bringyourknapsack/godspeed,False,Bringyourknapsack,run-it-up666-follower-special.mp3,2.94
1,Life Of A Hentai Dude,https://i1.sndcdn.com/avatars-aPXogjt1jfUkpBfY...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/korewamoemoemoe/life-of...,False,Korewamoemoemoe,a-day-in-the-park.mp3,1.83
2,Amagami Ss Nakata Sae Ed,https://i1.sndcdn.com/artworks-000058281299-wg...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/bafaloon/amagami-ss-nak...,False,Bafaloon,tofuku-taipein3kos-remix.mp3,4.00
3,Going On,https://i1.sndcdn.com/avatars-XuMm9HkRL3viyzx2...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/kshiraki/going-on,False,Kshiraki,san-holo-voices-in-my-head-last-island-remix.mp3,2.94
4,Heisoku No Kakudai,https://i1.sndcdn.com/artworks-49w79Us6tJS0lUM...,https://github.com/ADGVLOGS/ADGMusicPlayer/blo...,https://soundcloud.com/user-909844975/heisoku-...,False,User 909844975,move.mp3,3.50


#### Converting dataframe to .json responses

In [24]:
def ConvertToJsonObject():
    data = []
    data.append('tracks: [')
    for index,row in dfmusic.iterrows():
        data.append("{")
        data.append("           name: \"{0}\",".format(row['name']))
        data.append("           artist: \"{0}\",".format(row['artist']))
        data.append("           cover: \"{0}\",".format(row['cover']))
        data.append("           source: \"{0}\",".format(row['source']))
        data.append("           url: \"{0}\",".format(row['url']))
        data.append("           favorited: \"{0}\",".format('false'))
        data.append("},")
        
    data.append('],')
    with open('responses.json', 'w') as filehandle:
        for listitem in data:
            filehandle.write('%s\n' % listitem)       
    print("filesaved to responses.json")

ConvertToJsonObject()

filesaved to responses.json


##### Copyright (c) ADGSTUDIOS 2021